In [1]:
import json
import os
import time

import requests

from utils import create_from_file, get_optimized_routes, load_generator

In [2]:
with open("data/cuopt-payload.json", "r") as f:
    data = json.load(f)
    payload = {"data": data}

# NVCF Deployment

In [3]:
NGC_API_KEY = os.getenv("NGC_API_KEY")
invoke_url = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/functions/c8b8b8c0-c2a1-427d-abba-a078dc68050e"
fetch_url_format = "https://api.nvcf.nvidia.com/v2/nvcf/pexec/status/"
fetch_url = invoke_url
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {NGC_API_KEY}",
    "NVCF-POLL-SECONDS": "5",
}

In [4]:
response = get_optimized_routes(invoke_url, payload, headers)
response["response"]

2025-04-17 21:07:09,746 - INFO - Polling NVCF Request ID: b5c85d13-b881-42cc-b338-5aeb263db02b
2025-04-17 21:07:09,747 - INFO - Initial response status: 200
2025-04-17 21:07:09,748 - INFO - Request completed: duration=1.97s, polls=0, status=200, size=797 bytes


{'solver_response': {'status': 0,
  'num_vehicles': 2,
  'solution_cost': 2.0,
  'objective_values': {'cost': 2.0},
  'vehicle_data': {'veh-1': {'task_id': ['Break', 'Task-A'],
    'arrival_stamp': [1.0, 2.0],
    'type': ['Break', 'Delivery'],
    'route': [1, 1]},
   'veh-2': {'task_id': ['Depot', 'Break', 'Task-B', 'Depot'],
    'arrival_stamp': [2.0, 2.0, 4.0, 5.0],
    'type': ['Depot', 'Break', 'Delivery', 'Depot'],
    'route': [0, 0, 2, 0]}},
  'dropped_tasks': {'task_id': [], 'task_index': []}},
 'perf_times': {'etl_time': 0.033437252044677734,
  'solver_run_time': 0.6057307720184326}}

In [5]:
responses = load_generator(
    num_requests=10,
    concurrency=1,
    url=invoke_url,
    payloads=[payload],
    timeout=30,
    request_delay=0.05,
    headers=headers,
)

2025-04-17 21:07:09,765 - INFO - Starting load generator with 10 requests and concurrency 1
2025-04-17 21:07:09,768 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,769 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,770 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,771 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,772 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,773 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09,774 - ERROR - Request failed: get_optimized_routes() got an unexpected keyword argument 'fetch_url_format'
2025-04-17 21:07:09

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

Here we are demonstrating this performance on a large popular academic [dataset by Gehring & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/). These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems. Each problem instance has an associated best known solution, the one we will measure against is shown below

API Reference: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

In [6]:
import os

import msgpack
import numpy as np
import pandas as pd
from scipy.spatial import distance

In [7]:
homberger_1000_file = "data/C1_10_1.TXT"
best_known_solution = {"n_vehicles": 100, "cost": 42478.95}

In [8]:
orders, vehicle_capacity, n_vehicles = create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0] - 1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)

/workspace/utils.py:268: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)


Number of locations          :  1000
Number of vehicles available :  250
Capacity of each vehicle     :  200


## Cost Matrix

In [9]:
cuopt_problem_data = {}

coords = list(zip(orders["xcord"].to_list(), orders["ycord"].to_list()))

cost_matrix = (
    pd.DataFrame(distance.cdist(coords, coords, "euclidean"))
    .astype(np.float32)
    .values.tolist()
)

## Set Cost Matrix

In [10]:
cuopt_problem_data["cost_matrix_data"] = {"data": {"0": cost_matrix}}

## Set Fleet Data

In [11]:
vehicle_locations = [[0, 0]] * n_vehicles

cuopt_problem_data["fleet_data"] = {
    "vehicle_locations": vehicle_locations,
    "capacities": [[vehicle_capacity] * n_vehicles],
}

## Set Task Data

In [12]:
cuopt_problem_data["task_data"] = {
    "task_locations": orders["vertex"].values.tolist(),
    "demand": [orders["demand"].values.tolist()],
    "task_time_windows": [
        list(i)
        for i in list(
            zip(
                orders["earliest_time"].values.tolist(),
                orders["latest_time"].values.tolist(),
            )
        )
    ],
    "service_times": orders["service_time"].values.tolist(),
}

cuopt_problem_data["solver_config"] = {"time_limit": 15.0}

## Helper functions to solve and process the output

In [13]:
def solution_eval(vehicles, cost, best_known_solution):

    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(
        f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution"
    )
    print(
        f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n"
    )
    print(f"- In addition cuOpt provides a solution cost of {cost}")
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

## Upload Asset for NVCF

In [14]:
asset_headers = {
    "Authorization": f"Bearer {NGC_API_KEY}",
    "accept": "application/json",
    "Content-Type": "application/json",
}

asset_data = {
    "contentType": "application/octet-stream",
    "description": "Optimization-data",
}

asset_response = requests.post(
    "https://api.nvcf.nvidia.com/v2/nvcf/assets", headers=asset_headers, json=asset_data
)

In [15]:
pickled_cuopt_data = msgpack.dump(cuopt_problem_data, open("problem_data.mpk", "wb"))

In [16]:
asset_submit_headers = {
    "Content-Type": "application/octet-stream",
    "x-amz-meta-nvcf-asset-description": "Optimization-data",
}

with open("problem_data.mpk", "rb") as f:
    response = requests.put(
        asset_response.json()["uploadUrl"], headers=asset_submit_headers, data=f
    )

In [17]:
headers["NVCF-INPUT-ASSET-REFERENCES"] = asset_response.json()["assetId"]

## Get Optimized Results
Update solver config and test different run-time

_1 Minute Time Limit_

> **Note**: due to the large amount of data network transfer time can exceed the requested solve time.

In [18]:
no_data = {"data": None, "client_version": "custom"}

# Solve the problem
solver_response = get_optimized_routes(invoke_url, no_data, headers, fetch_url_format, timeout=600, request_delay=1.0)
solver_response["response"]["perf_times"]

TypeError: get_optimized_routes() got multiple values for argument 'timeout'

In [ ]:
# Process returned data
solver_resp = solver_response["response"]["solver_response"]

# Evaluation
solution_eval(
    solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution
)